# Proyecto Sprint 10

## Introducción

En este proyecto se realiza la creación de un modelo que logre predecir si un cliente dejará el banco Beta Bank y se tratara de obtener un máximo valor F1 posible, de al menos 0.59.

También se procesarán los datos para lidiar con los problemas de datos contenían características categóricas, desequilibrio de clases y se evaluara el rendimiento del modelo mediante las metricas Recall, Precision, F1 y AUC-ROC.

## 1.- Descarga y revisión de datos

In [2]:
#Importacion de librerias
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    average_precision_score,
    classification_report)
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV

In [5]:
#Importación de información
data = pd.read_csv('Churn.csv')

In [6]:
#Revisamos la información que tenemos, analizamos los tipos de datos que tiene cada columna, valores ausentes.
display(
    data.info(), #Revisamos el tipo de datos que tenemos, el tamaño del data, las columnas
    data.describe(), #Información de nuestros datos
    data.isna().mean() #Vemos la % de datos nullos en las columnas
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


None

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


RowNumber          0.0000
CustomerId         0.0000
Surname            0.0000
CreditScore        0.0000
Geography          0.0000
Gender             0.0000
Age                0.0000
Tenure             0.0909
Balance            0.0000
NumOfProducts      0.0000
HasCrCard          0.0000
IsActiveMember     0.0000
EstimatedSalary    0.0000
Exited             0.0000
dtype: float64

Obtuvimos que tenemos un 9.09 % de valores nulos en la columna "Ternure"

### Imputación de datos

Como tenemos valores nulos en nuestros datos, procedemos a imputarlos

In [7]:
#Imputación de datos numericos

#Con KNNImputer predecimos los valores faltantes en función de los valores más similares dentro del data set
imputer = KNNImputer(n_neighbors=3, weights="uniform") #Creamos el objeto
df_imputed = pd.DataFrame(imputer.fit_transform(data[['Tenure']])) #Aplicamos a la columna "Ternure"
data['Tenure'] = df_imputed #Lo reemplazamos en el dataframe original
print(data['Tenure'].isna().mean()) #Revisamos una vez más para ver que ya no haya nulos

0.0


### Transformación de datos


In [8]:
#Identificamos nuestras columnas numericas y categoricas
num_col = ['RowNumber', 'CustomerId','CreditScore','Age','Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember','EstimatedSalary']
cat_not_ord_col = ['Surname','Geography','Gender']

#### Estandarización de datos para datos numericos

In [9]:
scaler_standard = StandardScaler()
data_standard = pd.DataFrame(scaler_standard.fit_transform(data[num_col]), columns=data[num_col].columns)
print(data_standard)

      RowNumber  CustomerId  CreditScore       Age        Tenure   Balance  \
0     -1.731878   -0.783213    -0.326221  0.293517 -1.086170e+00 -1.225848   
1     -1.731531   -0.606534    -0.440036  0.198164 -1.448505e+00  0.117350   
2     -1.731185   -0.995885    -1.536794  0.293517  1.087844e+00  1.333053   
3     -1.730838    0.144767     0.501521  0.007457 -1.448505e+00 -1.225848   
4     -1.730492    0.652659     2.063884  0.388871 -1.086170e+00  0.785728   
...         ...         ...          ...       ...           ...       ...   
9995   1.730492   -1.177652     1.246488  0.007457  8.369869e-04 -1.225848   
9996   1.730838   -1.682806    -1.391939 -0.373958  1.812515e+00 -0.306379   
9997   1.731185   -1.479282     0.604988 -0.278604  7.255082e-01 -1.225848   
9998   1.731531   -0.119356     1.256835  0.293517 -7.238342e-01 -0.022608   
9999   1.731878   -0.870559     1.463771 -1.041433 -3.218187e-16  0.859965   

      NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary

#### One-Hot Encoding para datos categoricos

In [10]:
# Aplicamos One-Hot Encoding a la columna 'Ciudad'
onehot_encoder = OneHotEncoder(sparse_output=False, drop='first')  # `drop='first'` para evitar multicolinealidad
data_one_hot = onehot_encoder.fit_transform(data[cat_not_ord_col])

# Convertimos la salida en un DataFrame y lo unimos al original
data_one_hot = pd.DataFrame(data_one_hot, columns=onehot_encoder.get_feature_names_out(cat_not_ord_col))
data_clean = pd.concat([data, data_one_hot], axis=1).drop(columns=cat_not_ord_col)

print(data_clean) #Obtenemos un data set listo para analizar pero que sigue desbalanceado

      RowNumber  CustomerId  CreditScore  Age    Tenure    Balance  \
0             1    15634602          619   42   2.00000       0.00   
1             2    15647311          608   41   1.00000   83807.86   
2             3    15619304          502   42   8.00000  159660.80   
3             4    15701354          699   39   1.00000       0.00   
4             5    15737888          850   43   2.00000  125510.82   
...         ...         ...          ...  ...       ...        ...   
9995       9996    15606229          771   39   5.00000       0.00   
9996       9997    15569892          516   35  10.00000   57369.61   
9997       9998    15584532          709   36   7.00000       0.00   
9998       9999    15682355          772   42   3.00000   75075.31   
9999      10000    15628319          792   28   4.99769  130142.79   

      NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  ...  \
0                 1          1               1        101348.88  ...   
1              

## 2.- Examinación del equilibrio de clases

#### Revisamos el equilibrio entre clases

In [11]:
data_clean['Exited'].value_counts(1)

Exited
0    0.7963
1    0.2037
Name: proportion, dtype: float64

#### Entrenamos un modelo sin estandarizar los datos

In [12]:
#MODELO SIN BALANCEAR

#Segmentación de datos
features = data_clean.drop(['Exited'], axis=1)
target = data_clean['Exited']

# Dividir los datos en entrenamiento + validación (80%) y prueba (20%)
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.2, random_state=42)
# Modelo en dataset 
model = RandomForestClassifier()
model.fit(features_train, target_train)
predict = model.predict(features_valid)
y_prob = model.predict_proba(features_valid)[:, 1]

In [16]:
#Cálculo de métricas
accuracy = accuracy_score(target_valid, predict)
precision = precision_score(target_valid, predict)
recall = recall_score(target_valid, predict)
f1 = f1_score(target_valid, predict)
roc_auc = roc_auc_score(target_valid, y_prob)
average_precision = average_precision_score(target_valid, y_prob)

# 📌 Mostramos los valores de las métricas
print(f"🔹 Accuracy: {accuracy:.2f}")
print(f"🔹 Precision: {precision:.2f}")
print(f"🔹 Recall: {recall:.2f}")
print(f"🔹 F1-Score: {f1:.2f}")
print(f"🔹 ROC-AUC: {roc_auc:.2f}")
print(f"🔹 Average Precision (AP): {average_precision:.2f}")

🔹 Accuracy: 0.86
🔹 Precision: 0.79
🔹 Recall: 0.36
🔹 F1-Score: 0.50
🔹 ROC-AUC: 0.85
🔹 Average Precision (AP): 0.66


### Hallazgos:

Accuracy el 85% de las predicciónes coincidieron con la validación.
De las respuestas positivas en Precision, obtuvimos que el 81% de las respuestas positivas fue correcta.
Con recall obtuvimos que se detectaron un 34% de casos realmente positivos.
Hay un balance entre recall y precision de un 48%
Tiene una AP de 66% que no esta tan mal

## 3.- Mejoración de calidad del modelo

#### Equilibrio de clases

In [17]:
#Balanceo de datos: Aplicando Undersampling y luego Oversampling
undersample = RandomUnderSampler(sampling_strategy=0.3, random_state=42)  # Reducir clase mayoritaria
X_under, y_under = undersample.fit_resample(data_clean, data_clean['Exited'])

oversample = SMOTE(sampling_strategy=0.67, random_state=42)  # Aumentar clase minoritaria
X_resampled, y_resampled = oversample.fit_resample(X_under, y_under)

y_resampled.value_counts(1)

Exited
0    0.598818
1    0.401182
Name: proportion, dtype: float64

##### Segmentación de datos balanceados

In [18]:
# Dividir los datos en entrenamiento + validación (80%) y prueba (20%)
features_train, features_valid, target_train, target_valid = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

### Busqueda de mejor conjunto de parámetros con GridSearchCV

In [19]:
# Definir el modelo base
modelo_dec_tre = DecisionTreeClassifier(random_state=42)

# Definir hiperparámetros a evaluar
grid_params = {
    'max_depth': [3, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'criterion': ['gini', 'entropy']
}

# Configurar GridSearchCV con la métrica f1-score para datos desbalanceados
grid_search = GridSearchCV(modelo_dec_tre, grid_params, scoring='f1', cv=5, n_jobs=-1)
grid_search.fit(features_train, target_train)

# Mejor modelo y evaluación
y_pred = grid_search.best_estimator_.predict(features_valid)
print("Mejores hiperparámetros:", grid_search.best_params_)
print("Reporte de clasificación:")
print(classification_report(target_valid, y_pred))


Mejores hiperparámetros: {'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 2}
Reporte de clasificación:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1387
           1       1.00      1.00      1.00       881

    accuracy                           1.00      2268
   macro avg       1.00      1.00      1.00      2268
weighted avg       1.00      1.00      1.00      2268



## 4.- Prueba final

In [24]:
y_prob = grid_search.predict_proba(features_valid)[:, 1]

In [25]:
#Cálculo de métricas
accuracy = accuracy_score(target_valid, y_pred)
roc_auc = roc_auc_score(target_valid, y_prob)
average_precision = average_precision_score(target_valid, y_prob)

# 📌 Mostramos los valores de las métricas
print(f"🔹 Accuracy: {accuracy:.2f}")
print(f"🔹 ROC-AUC: {roc_auc:.2f}")
print(f"🔹 Average Precision (AP): {average_precision:.2f}")

🔹 Accuracy: 1.00
🔹 ROC-AUC: 1.00
🔹 Average Precision (AP): 1.00


## Conclusión final

Para lograr entrenar adecuadamente nuestro modelo, primero hay que preparar los datos, hay que escoger correctamente como vamos a rellenar los valores faltantes, ya que dependiendo de lo que escojamos los datos podrían ser más preciso. En mi caso escogí KNNImputer ya que me parece mejor escoger datos que esten más cerca unos de otros. Después de eso tenemos que transformar los datos para que nuestro modelo no le de más valor a ciertos datos o ciertas columnas por lo que hay que estandarizar los valores numericos y transformar nuestros valores categoricos a etiquetas o valores numericos dependiendo si tienen orden o no las categorias, como niveles escolares y cosas así.

Una vez teniendo nuestros datos listos para que el modelo pueda aprender, podemos entrenarlo pero aun así hay que tomar en cuenta el balance de clases, ya que podría aprender mal o interpretar mal los datos en ciertos casos y causando más falsos resultados. Por lo que en ciertas ocaciones hay que disminuir la clase mayoritaria o incrementar la minoritaria.

Finalmente cuando entrenamos nuestro modelo balanceado y con los trasnformados hay que tomar en cuenta los parametros de entrenamiento del modelo, ya que dependiendo la combinación de estos puede incrementar la precisión de nuestro modelo.